In [1]:
import io
from pathlib import Path

from google.cloud import bigquery


ndjson = io.BytesIO(b"""{"ID":0,"BedrijfstakkenBranches":"300010","Perioden":"2005KW01","IndexcijfersOmzet_1":"74.0","OmzetontwikkelingTOVEenJaarEerder_2":"."}
{"ID":1,"BedrijfstakkenBranches":"300010","Perioden":"2005KW02","IndexcijfersOmzet_1":"84.0","OmzetontwikkelingTOVEenJaarEerder_2":"."}
{"ID":2,"BedrijfstakkenBranches":"300010","Perioden":"2005KW03","IndexcijfersOmzet_1":"83.0","OmzetontwikkelingTOVEenJaarEerder_2":"."}
{"ID":3,"BedrijfstakkenBranches":"300010","Perioden":"2005KW04","IndexcijfersOmzet_1":"95.2","OmzetontwikkelingTOVEenJaarEerder_2":"."}
{"ID":4,"BedrijfstakkenBranches":"300010","Perioden":"2005JJ00","IndexcijfersOmzet_1":"84.0","OmzetontwikkelingTOVEenJaarEerder_2":"."}
{"ID":5,"BedrijfstakkenBranches":"300010","Perioden":"2006KW01","IndexcijfersOmzet_1":"82.2","OmzetontwikkelingTOVEenJaarEerder_2":"11.2"}
{"ID":6,"BedrijfstakkenBranches":"300010","Perioden":"2006KW02","IndexcijfersOmzet_1":"91.7","OmzetontwikkelingTOVEenJaarEerder_2":"9.2"}
{"ID":7,"BedrijfstakkenBranches":"300010","Perioden":"2006KW03","IndexcijfersOmzet_1":"91.2","OmzetontwikkelingTOVEenJaarEerder_2":"9.9"}
{"ID":8,"BedrijfstakkenBranches":"300010","Perioden":"2006KW04","IndexcijfersOmzet_1":"105.5","OmzetontwikkelingTOVEenJaarEerder_2":"10.8"}
{"ID":9,"BedrijfstakkenBranches":"300010","Perioden":"2006JJ00","IndexcijfersOmzet_1":"92.6","OmzetontwikkelingTOVEenJaarEerder_2":"10.3"}
{"ID":10,"BedrijfstakkenBranches":"300010","Perioden":"2007KW01","IndexcijfersOmzet_1":"91.0","OmzetontwikkelingTOVEenJaarEerder_2":"10.7"}
""")

csv = io.BytesIO(b"""straat,woonplaats,minPC4,maxPC4,divPC4,aantPC4
Blondeelstraat,Rotterdam,3067,9999,6932,2
Molenweg,Zevenhuizen,2761,9354,6593,2
De Kamp	Laren,1251,7245,5994,2
Kloosterweg,Laren,1251,7245,5994,2
Nieuweweg,Laren,1251,7245,5994,2
""")


In [ ]:

dataset_id = 'test'
table_id = 'test_ndjson'
client = bigquery.Client()

table_ref = client.dataset(dataset_id).table(table_id)
job_config = bigquery.LoadJobConfig()
job_config.source_format = bigquery.SourceFormat.NEWLINE_DELIMITED_JSON
# job_config.skip_leading_rows = 1
job_config.autodetect = True

with ndjson as source_file:
    job = client.load_table_from_file(source_file, table_ref, job_config=job_config)

job.result()  # Waits for table load to complete.

print("Loaded {} rows into {}:{}.".format(job.output_rows, dataset_id, table_id))

In [1]:
from prefect.tasks.gcp.bigquery import BigQueryLoadFile
from prefect import Flow

dataset_id = 'test'
table_id = 'test_ndjson'
file = 'test_ndjson'

load = BigQueryLoadFile()
with Flow("Load file") as flow:
    l = load(file='test.ndjson', dataset_id=dataset_id, table=table_id, **{"source_format": "NEWLINE_DELIMITED_JSON"})

flow.run()

[2020-05-11 06:23:20] INFO - prefect.FlowRunner | Beginning Flow run for 'Load file'
[2020-05-11 06:23:20] INFO - prefect.FlowRunner | Starting flow run.
[2020-05-11 06:23:20] INFO - prefect.TaskRunner | Task 'BigQueryLoadFile': Starting task run...
[2020-05-11 06:23:25] INFO - prefect.TaskRunner | Task 'BigQueryLoadFile': finished task run for task with final state: 'Success'
[2020-05-11 06:23:25] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded


/Users/dkapitan/miniconda3/envs/timefflow/lib/python3.8/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/dkapitan/miniconda3/envs/timefflow/lib/python3.8/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accoun

<Success: "All reference tasks succeeded.">

In [10]:
job_config.to_api_repr()

{'load': {'sourceFormat': 'NEWLINE_DELIMITED_JSON', 'autodetect': True},
 'labels': {}}

In [11]:
**job_config.to_api_repr()

SyntaxError: invalid syntax (<ipython-input-11-9e420c5ef710>, line 1)

In [2]:
def test_oserror(file_path):
    try:
        with open(file_path, 'rb') as f:
            foo = f.read()
    except (OSError, TypeError) as error:
        print(f"Error trying to open {file_path}: {error!s}")


In [3]:
test_oserror('foo.txt')

Error trying to open foo.txt: [Errno 13] Permission denied: 'foo.txt'


In [ ]:
with open(1) as f:
    foo = f.read()

In [12]:
!ls

foo.txt                          test-local-secrets-prefect.ipynb
test-bigquery-load-file.ipynb    test.ndjson
test-cbs-81819.ndjson
